## Buggy project detect, creat and batch run

### Buggy code detect
Here we first detect the buggy project set for the buggy code

* `reference_folder`: `./vitis_examples/02_basic_examples_vhls_fir_filter` include full, vanilla project

* `buggy_folder`: `./vitis_examples_buggy/02_basic_examples_vhls_fir_filter` include buggy top modules

* `output folder` `./vitis_examples_out/02_basic_examples_vhls_fir_filter/.......` include different buggy projects

The initial structure of the folder should look like this:
```tree
.
├── bug_run.ipynb
├── vitis_examples
│   └── 02_basic_examples_vhls_fir_filter
├── vitis_examples_buggy
│   └── 02_basic_examples_vhls_fir_filter
└── vitis_examples_out
```

In [12]:
import os
parser_script_path = r'E:\VScode\HLS_LLM_Code\HLSGen\bug_playground\parser_single.py' # parser script
O_PATH = r"E:/VScode/NL2CPP_gpt_4o_simple"  # Replace with the desired output folder
S_PATH = r"E:\VScode\HLS_LLM_Code\HLSGen\origin_projects"  # Replace with the actual path to the reference folder
B_PATH = r"E:\VScode\HLS_LLM_Code\HLSGen\origin_projects"  # Replace with the actual path to the buggy folder
THREAD =   6
TCL_SCRIPT = "run_hls.tcl"
R_MODULE_NAMES = [
    "2mm",
    "3mm",
    "atax",
    "bicg",
    "correlation",
    "covariance",
    "floyd-warshall",
    "gemm",
    "gesummv",
    "heat-3d",
    "jacobi-2d",
    "mvt",
    "nussinov",
    "symm",
    "syr2k",
    "trmm"
]

In [13]:
import os
import shutil

def detect_buggy_projects(buggy_folder, reference_top_module_names):
    buggy_projects = {}
    
    # List of acceptable source file extensions
    acceptable_extensions = ['.c', '.cc', '.cpp', '.h', '.hpp']  # Add more extensions as needed
    
    # Iterate through the directories in the buggy folder
    for dirpath, dirnames, filenames in os.walk(buggy_folder):
        # Only check in subdirectories
        if dirpath != buggy_folder:
            buggy_files = []
            ref_names = []
            
            # Check for buggy files based on their naming conventions
            for filename in filenames:
                # Check if the filename ends with any of the acceptable extensions
                if any(filename.endswith(ext) for ext in acceptable_extensions):
                    # Extract the base name without the extension
                    base_name = os.path.splitext(filename)[0]
                    
                    # Check if the filename contains any of the reference top module names
                    for ref_name in reference_top_module_names:
                        if base_name.startswith("modified_"):
                            base_name = base_name[len("modified_"):]
                        if base_name.startswith(ref_name):  # Check if the base name starts with the ref_name
                            buggy_files.append(filename)
                            ref_names.append(ref_name)
                            break  # Stop after finding a match

            if buggy_files:
                project_name = os.path.basename(dirpath)  # Use the directory name as the project key
                buggy_projects[project_name] = {
                    'replaced_file': ref_names,  # Store the reference top module name
                    'files': buggy_files,
                    'path': dirpath  # Store the path to the directory
                }

    return buggy_projects

import os
import shutil

def assemble_buggy_projects(buggy_projects, reference_folder, output_folder, reference_top_module_names):
    if not buggy_projects:
        print("no buggy_projects")
        return

    for project, info in buggy_projects.items():
        buggy_files = info['files']
        dirpath = info['path']  # Get the directory path for the buggy files
        
        # Create a collection folder in output
        collection_folder = os.path.join(output_folder, project)
        os.makedirs(collection_folder, exist_ok=True)

        for buggy_file in buggy_files:
            # Determine the bug type from the buggy file name using the reference top module names
            bug_type = None  # Initialize bug_type as None
            for ref_name in reference_top_module_names:
                if buggy_file.startswith(ref_name):  # Check if buggy_file starts with the ref_name
                    # Extract the bug type correctly by removing the reference name
                    remainder = buggy_file[len(ref_name):].lstrip('_')  # Remove leading underscore if it exists
                    bug_type = remainder.split('.')[0]  # Extract bug type safely
                    break  # Stop after finding a match
            
            if bug_type is None:
                continue
            
            # Create a new subfolder for the bug type (buggy project folder)
            buggy_project_folder = os.path.join(collection_folder)
            # add an extra layer to indicate reference top module

            os.makedirs(buggy_project_folder, exist_ok=True)

            # Find the original reference project folder based on the buggy project name
            reference_project_folder = os.path.join(reference_folder, project)  # Assuming the original project is named similarly

            if not os.path.exists(reference_project_folder):
                continue

            # Copy all files from the original reference project
            for item in os.listdir(reference_project_folder):
                source_item = os.path.join(reference_project_folder, item)
                if os.path.isfile(source_item):
                    if(buggy_file == f"{ref_name}.cpp"):
                        shutil.copytree(reference_project_folder, buggy_project_folder, dirs_exist_ok=True)
            

            buggy_file_path = os.path.join(dirpath, buggy_file)  # Use the stored dirpath
            new_file_path = os.path.join(buggy_project_folder, buggy_file)
            print("buggy_file_path", buggy_file_path)
            print("new_file_path", new_file_path)
            # Replace the top module file with the buggy file
            shutil.copy(buggy_file_path, new_file_path)

            #

In [14]:
buggy_folder = B_PATH  # Replace with the actual path to the buggy folder
reference_folder = S_PATH  # Replace with the actual path to the reference folder
output_folder = O_PATH  # Replace with the desired output folder

reference_top_module_names = R_MODULE_NAMES  # Add more top module names as needed


In [15]:
# Detect buggy projects
buggy_projects = detect_buggy_projects(buggy_folder, reference_top_module_names)
print(buggy_projects)

# Check if there are any buggy projects detected

# Assemble the buggy projects
assemble_buggy_projects(buggy_projects, reference_folder, output_folder, reference_top_module_names)

{'2mm': {'replaced_file': ['2mm', '2mm', '2mm'], 'files': ['2mm.cpp', '2mm.h', '2mm_tb.cpp'], 'path': 'E:\\VScode\\HLS_LLM_Code\\HLSGen\\origin_projects\\2mm'}, '3mm': {'replaced_file': ['3mm', '3mm', '3mm'], 'files': ['3mm.cpp', '3mm.h', '3mm_tb.cpp'], 'path': 'E:\\VScode\\HLS_LLM_Code\\HLSGen\\origin_projects\\3mm'}, 'atax': {'replaced_file': ['atax', 'atax', 'atax', 'atax'], 'files': ['atax.cpp', 'atax.h', 'atax_tb copy.cpp', 'atax_tb.cpp'], 'path': 'E:\\VScode\\HLS_LLM_Code\\HLSGen\\origin_projects\\atax'}, 'bicg': {'replaced_file': ['bicg', 'bicg', 'bicg'], 'files': ['bicg.cpp', 'bicg.h', 'bicg_tb.cpp'], 'path': 'E:\\VScode\\HLS_LLM_Code\\HLSGen\\origin_projects\\bicg'}, 'correlation': {'replaced_file': ['correlation', 'correlation', 'correlation'], 'files': ['correlation.cpp', 'correlation.h', 'correlation_tb.cpp'], 'path': 'E:\\VScode\\HLS_LLM_Code\\HLSGen\\origin_projects\\correlation'}, 'covariance': {'replaced_file': ['covariance', 'covariance', 'covariance'], 'files': ['cova

In [16]:
import subprocess
import os
import concurrent.futures
import time
import threading
import psutil
from colorama import Fore, Style

# Shared structure to hold task status
task_status = {}
task_completed = threading.Event()  # Event to signal when all tasks are completed
final_results = {
    'DONE': [],
    'DONE_ERROR': [],
    'HANG_ERROR': []
}

def modify_tcl_script(tcl_script_path):
    """Modify the TCL script to change hls_exec value."""
    with open(tcl_script_path, 'r') as file:
        lines = file.readlines()

    # Modify the specific line
    for i, line in enumerate(lines):
        if "set hls_exec 0" in line:
            lines[i] = line.replace("set hls_exec 0", "set hls_exec 2")
            print(f"Modified {tcl_script_path}: set hls_exec changed to 2")
            break

    # Write the modified content back to the file
    with open(tcl_script_path, 'w') as file:
        file.writelines(lines)

def kill_csim_process():
    """Kill the csim.exe process if it exists."""
    for proc in psutil.process_iter():
        try:
            if proc.name() == "csim.exe":
                proc.kill()
                print(f"{Fore.RED}Killed csim.exe process.{Style.RESET_ALL}")
        except (psutil.NoSuchProcess, psutil.AccessDenied):
            continue

def update_task_status(project, elapsed_time, status):
    """Update the shared task status."""
    task_status[(project)] = (elapsed_time, status)

def run_vitis_hls_for_project(project_info):
    project, buggy_project_folder, logs_folder = project_info
    start_time = time.time()  # Start the timer
    dataset_tcl_script =  TCL_SCRIPT  # Set just the script name
    status = "RUNNING"

    if os.path.exists(os.path.join(buggy_project_folder, dataset_tcl_script)):
        print(f"{Fore.GREEN}Running Vitis HLS for project: {project}, TCL script: {dataset_tcl_script}{Style.RESET_ALL}")

        # Redirect stdout and stderr to a log file
        log_file_path = os.path.join(logs_folder, f"{project}_log.txt")
        with open(log_file_path, 'w') as log_file:
            try:
                # Run the Vitis HLS command by changing directory with a timeout
                process = subprocess.Popen(["E:/Vitis/Vitis_HLS/2022.1/bin/vitis_hls.bat", dataset_tcl_script], stdout=log_file, stderr=subprocess.STDOUT, cwd=buggy_project_folder)
                
                while True:
                    elapsed_time = time.time() - start_time
                    update_task_status(project, elapsed_time, status)

                    if process.poll() is not None:  # Process has finished
                        if process.returncode == 0:  # Successful run
                            status = "DONE"
                        else:  # Error during execution
                            status = "DONE ERROR"
                        update_task_status(project, elapsed_time, status)
                        final_results[status].append((project, elapsed_time))  # Record results
                        break
                    if elapsed_time > 300:  # 5 minutes timeout
                        print(f"{Fore.YELLOW}Timeout: Vitis HLS for {project} is hanging. Killing csim.exe and skipping this iteration.{Style.RESET_ALL}")
                        kill_csim_process()  # Kill csim.exe process
                        process.terminate()  # Terminate the Vitis HLS process
                        status = "HANG_ERROR"
                        update_task_status(project, elapsed_time, status)
                        final_results[status].append((project, elapsed_time))  # Record results
                        break
                    time.sleep(20)  # Refresh every 2 seconds
                
            except Exception as e:
                print(f"Error running Vitis HLS for {project}. Error: {e}")
                status = "DONE ERROR"
                update_task_status(project, elapsed_time, status)
                final_results[status].append((project, elapsed_time))  # Record results
    else:
        print(f"Warning: {dataset_tcl_script} not found in {buggy_project_folder}. Skipping compilation.")
        status = "DONE ERROR"
        update_task_status(project, elapsed_time, status)
        final_results[status].append((project, elapsed_time))  # Record results


def print_final_results(task_status):
    """Log the final results of the tasks."""
    done_count = 0
    done_error_count = 0
    hang_error_count = 0
    
    with open("run_results.log", "w") as log_file:
        log_file.write("===========================================\n")
        for (project), (elapsed_time, status) in task_status.items():
            if status == "DONE":
                log_file.write(f"@DONE      :  {project} , Executed time: {elapsed_time:.2f} seconds\n")
                done_count += 1
            elif status == "DONE ERROR":
                log_file.write(f"@DONE_ERROR: {project} , Executed time: {elapsed_time:.2f} seconds\n")
                done_error_count += 1
            elif status == "HANG_ERROR":
                log_file.write(f"@HANG_ERROR: {project} , Executed time: {elapsed_time:.2f} seconds\n")
                hang_error_count += 1
        
        log_file.write("===========================================\n")
        log_file.write(f"Total tasks: {done_count + done_error_count + hang_error_count}\n")
        log_file.write(f"@DONE tasks total: {done_count}\n")
        log_file.write(f"@DONE_ERROR tasks total: {done_error_count}\n")
        log_file.write(f"@HANG_ERROR tasks total: {hang_error_count}\n")
        log_file.write("===========================================\n")



def clear_output():
    """Clear the console output."""
    os.system('cls' if os.name == 'nt' else 'clear')

def print_all_task_statuses():
    """Print the status of all tasks."""
    while not task_completed.is_set():  # Continue until all tasks are completed
        if not task_status:
            continue
        
        # Clear the previous output
        clear_output()
        
        print("\nCurrent Task status:")
        
        # Make a copy of the task_status dictionary for safe iteration
        current_statuses = task_status.copy()
        
        done_count = 0
        done_error_count = 0
        hang_error_count = 0
        
        for (project), (elapsed_time, status) in current_statuses.items():
            if status == "DONE":
                done_count += 1
                print(f"Task for {Fore.BLUE}{project}{Style.RESET_ALL} Executed time: {elapsed_time:.2f} seconds - {Fore.GREEN}DONE{Style.RESET_ALL}")
            elif status == "DONE ERROR":
                done_error_count += 1
                print(f"Task for {Fore.BLUE}{project}{Style.RESET_ALL}  Executed time: {elapsed_time:.2f} seconds - {Fore.YELLOW}DONE ERROR{Style.RESET_ALL}")
            elif status == "HANG_ERROR":
                hang_error_count += 1
                print(f"Task for {Fore.BLUE}{project}{Style.RESET_ALL}  Executed time: {elapsed_time:.2f} seconds - {Fore.RED}HANG_ERROR{Style.RESET_ALL}")
            else:
                print(f"Task for {Fore.BLUE}{project}{Style.RESET_ALL}  Executed time: {elapsed_time:.2f} seconds - {Fore.WHITE}{status}{Style.RESET_ALL}")
        
        print("-" * 60)  # Separator for clarity
        print(f"Total tasks: {done_count + done_error_count + hang_error_count}")
        print(f"@DONE tasks total: {done_count}")
        print(f"@DONE_ERROR tasks total: {done_error_count}")
        print(f"@HANG_ERROR tasks total: {hang_error_count}")
        print("=" * 60)  # Separator for clarity
        
        time.sleep(20)  # Refresh every 2 seconds


In [17]:
import os
import threading
import concurrent.futures


def run_vitis_hls(buggy_projects, output_folder):
    if not buggy_projects:
        print("No buggy projects to compile.")
        return

    project_infos = []

    for project, info in buggy_projects.items():
        project_path = os.path.join(output_folder, project)

        ref_path = os.path.join(project_path)
        logs_folder = os.path.join(ref_path, "logs")
        os.makedirs(logs_folder, exist_ok=True)

        buggy_project_folder = os.path.join(ref_path)
        project_infos.append((f"{project}", buggy_project_folder, logs_folder))

    # Run task status printer in a separate thread
    status_thread = threading.Thread(target=print_all_task_statuses, daemon=True)
    status_thread.start()

    # Run Vitis HLS in parallel
    with concurrent.futures.ThreadPoolExecutor(max_workers=THREAD) as executor:
        futures = [executor.submit(run_vitis_hls_for_project, project_info) for project_info in project_infos]
        concurrent.futures.wait(futures)  # Wait for all tasks to complete

    # Assuming `task_completed` is a threading.Event() instance declared elsewhere
    task_completed.set()  # Signal that all tasks are completed
    

In [18]:
# Example usage
# Assuming `buggy_projects` is already populated from the previous step
run_vitis_hls(buggy_projects, output_folder)

Running Vitis HLS for project: 2mm, TCL script: run_hls.tcl
Running Vitis HLS for project: 3mm, TCL script: run_hls.tcl
Running Vitis HLS for project: atax, TCL script: run_hls.tcl
Running Vitis HLS for project: bicg, TCL script: run_hls.tcl
Running Vitis HLS for project: correlation, TCL script: run_hls.tcl
Running Vitis HLS for project: covariance, TCL script: run_hls.tcl

Current Task status:
Task for 2mm  Executed time: 0.06 seconds - RUNNING
Task for 3mm  Executed time: 0.04 seconds - RUNNING
Task for atax  Executed time: 0.02 seconds - RUNNING
Task for correlation  Executed time: 0.01 seconds - RUNNING
------------------------------------------------------------
Total tasks: 0
@DONE tasks total: 0
@DONE_ERROR tasks total: 0
@HANG_ERROR tasks total: 0

Current Task status:
Task for 2mm  Executed time: 20.07 seconds - RUNNING
Task for 3mm  Executed time: 20.05 seconds - RUNNING
Task for atax  Executed time: 20.03 seconds - RUNNING
Task for correlation  Executed time: 20.02 seconds 

In [19]:
print_final_results(task_status)

In [20]:
import os
import shutil
import glob
import subprocess
import sys

def get_file_size_mb(file_path):
    return os.path.getsize(file_path) / (1024 * 1024)  # Convert bytes to MB


def copy_log_files(O_PATH):
    input_folder = O_PATH
    output_folder = f"logs_{os.path.basename(O_PATH)}"

    # Check if the input folder exists
    if not os.path.isdir(input_folder):
        print("Error: Input folder does not exist.")
        return None

    # Check if the output folder exists, if not, create it
    if not os.path.isdir(output_folder):
        print("Output folder does not exist. Creating it now...")
        os.makedirs(output_folder, exist_ok=True)

    # Find all files ending with '_log.txt' in the input folder and subfolders
    # and copy them to the output folder
    #for file in glob.glob(os.path.join(input_folder, '**', '*_log.txt'), recursive=True):
    #    shutil.copy(file, output_folder)
    for file in glob.glob(os.path.join(input_folder, '**', '*_log.txt'), recursive=True):
        file_size_mb = get_file_size_mb(file)
        if file_size_mb <= 100:  # Only copy if file size is 100MB or less
            shutil.copy(file, output_folder)
        else:
            print(f"Skipped {file} as its size ({file_size_mb:.2f} MB) exceeds 100 MB")

    print("All files have been copied successfully.")
    return output_folder


def run_parser(parser_path, input_folder, json=True, csv=True):
    # Generate output file names
    base_name = f"result_{os.path.basename(input_folder)}"
    csv_path = f"{base_name}.csv" if csv else None
    json_path = f"{base_name}.json" if json else None

    command = [sys.executable, parser_path]
    
    if csv and csv_path:
        command.extend(['-c', csv_path])
    if json and json_path:
        command.extend(['-j', json_path])
    
    command.append(input_folder)
    
    result = subprocess.run(command, capture_output=True, text=True)
    
    print("STDOUT:")
    print(result.stdout)
    
    print("STDERR:")
    print(result.stderr)
    
    return result


In [21]:
# First, copy the log files
output_folder = copy_log_files(O_PATH)

Output folder does not exist. Creating it now...
All files have been copied successfully.


In [22]:

if output_folder:
    # Now run the parser on the copied log files
    result = run_parser(parser_script_path, output_folder, json=False, csv=True)

    # Print the generated file names
    print(f"CSV output: result_{os.path.basename(output_folder)}.csv")
    result = run_parser(parser_script_path, output_folder, json=True, csv=False)
    print(f"JSON output: result_{os.path.basename(output_folder)}.json")

STDOUT:
Processing log file: logs_NL2CPP_gpt_4o_simple\2mm_log.txt
Matched names: 2mm 2mm
Processing project: 2mm_2mm
Processing log file: logs_NL2CPP_gpt_4o_simple\3mm_log.txt
Matched names: 3mm 3mm
Processing project: 3mm_3mm
Processing log file: logs_NL2CPP_gpt_4o_simple\atax_log.txt
Matched names: atax atax
Processing project: atax_atax
Processing log file: logs_NL2CPP_gpt_4o_simple\bicg_log.txt
Matched names: bicg bicg
Processing project: bicg_bicg
Processing log file: logs_NL2CPP_gpt_4o_simple\correlation_log.txt
Matched names: correlation correlation
Processing project: correlation_correlation
Processing log file: logs_NL2CPP_gpt_4o_simple\covariance_log.txt
Matched names: covariance covariance
Processing project: covariance_covariance
Processing log file: logs_NL2CPP_gpt_4o_simple\floyd-warshall_log.txt
Matched names: floyd-warshall floyd-warshall
Processing project: floyd-warshall_floyd-warshall
Processing log file: logs_NL2CPP_gpt_4o_simple\gemm_log.txt
Matched names: gemm ge